In [1]:
import os

from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv, find_dotenv

DOTENV_PATH = find_dotenv()
if DOTENV_PATH:
    load_dotenv(DOTENV_PATH)

os.environ
dataservice_db_url = os.getenv("KF_DATASERVICE_DB_URL")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_GATE")

url = dataservice_db_url.format(DB_USER=db_user,
                                DB_GATE=db_pass)

engine = create_engine(url)

In [2]:
portal_studies_df = pd.read_table("studies_on_portal.tsv")
study_list = portal_studies_df['studies_on_portal'].to_list()
study_list_str = ','.join([f"'{item}'" for item in study_list])
study_list_str

"'SD_0TYVY1TW','SD_1P41Z782','SD_46RR9ZR6','SD_46SK55A3','SD_6FPYJQBR','SD_7NQ9151J','SD_8Y99QZJJ','SD_9PYZAHHE','SD_AQ9KVN5P','SD_B8X3C1MX','SD_BHJXBDQK','SD_DK0KRWK8','SD_DYPMEHHF','SD_DZ4GPQX6','SD_DZTB5HRR','SD_FFVQ3T38','SD_JWS3V24D','SD_NMVV8A1Y','SD_P445ACHV','SD_PET7Q6F2','SD_PREASA7S','SD_R0EPRSGS','SD_RM8AFW0R','SD_VTTSHWV4','SD_W0V965XZ','SD_YGVA0E1C','SD_YNSSAPHE','SD_Z6MWD3H0','SD_ZFGDG5YS','SD_ZXJFFMEF','SD_Z0D9N23X','SD_2CEKQ05V','SD_54G4WG4R','SD_JK4Z4T6V','SD_W6FWTD8A'"

In [3]:
query = f"""
select count(*)
from study s
join participant p on p.study_id = s.kf_id
where s.kf_id  in
({study_list_str})
and 
p.visible = true
"""

part_count_df = pd.read_sql(query,engine)

part_count_df

,count
0,29809


In [4]:
query = f"""
select count(*)
from study s
join participant p on p.study_id = s.kf_id
join biospecimen b on b.participant_id  = p.kf_id 
where s.kf_id  in
({study_list_str})
and 
p.visible = true
and
b.visible = true
"""

biospecimen_count_df = pd.read_sql(query,engine)

biospecimen_count_df

,count
0,40706


In [5]:
query = f"""
select count(*)
from study s
join participant p on p.study_id = s.kf_id
join biospecimen b on b.participant_id  = p.kf_id 
join biospecimen_genomic_file bgf on bgf.biospecimen_id = b.kf_id
join genomic_file gf on gf.kf_id = bgf.genomic_file_id
join file_metadata.indexd_scrape is2 on is2.did = gf.latest_did 
where s.kf_id  in
({study_list_str})
and 
p.visible = true
and
b.visible = true
and
gf.visible = true
"""

file_count_df = pd.read_sql(query,engine)

file_count_df

,count
0,369794


In [6]:
engine.dispose()